<a href="https://colab.research.google.com/github/shaoyinguo-portfolio/yield-triggers/blob/main/yield_triggers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Configs

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact, chi2_contingency

N_DAYS = 90
N_PRODUCTS = 2
N_SITES = 2
N_LINKS = 3
N_SHIFTS = 6
N_LOTS = 1000
N_DEFECTMODES = 5
QTY_MEAN = 1000
QTY_STD = 100
LOSS_MEAN = 0
LOSS_STD = 2

P_THRESHOLD = 0.05

REF_GROUP = ['Product']
TRIGGER_GROUP = ['Site', 'Link', 'WeekDay', 'Shift']

##Simulated Data Generation

In [24]:
np.random.seed(42)

df = pd.DataFrame(np.random.rand(N_LOTS,5), columns=['DaysFromNow', 'Product', 'Site', 'Link', 'Shift']) * np.array([N_DAYS, N_PRODUCTS, N_SITES, N_LINKS, N_SHIFTS])
df = df.astype(int).sort_values(['DaysFromNow'], ascending=False)
df['WeeksFromNow'] = df['DaysFromNow'] // 7
df['WeekDay'] = df['DaysFromNow'] % 7

DEFECT_COLUMNS = [f'Defect_{i}' for i in range(N_DEFECTMODES)]

DATA_COLUMNS = ['Qty'] + DEFECT_COLUMNS

df2 = pd.DataFrame(np.random.randn(N_LOTS,1 + N_DEFECTMODES), columns=DATA_COLUMNS) * np.array([QTY_STD] + [LOSS_STD]*N_DEFECTMODES) + np.array([QTY_MEAN] + [LOSS_STD]*N_DEFECTMODES)
df2 = df2.astype(int).clip(lower=0)

df = pd.concat([df, df2], axis=1)

df.head(10)

,DaysFromNow,Product,Site,Link,Shift,WeeksFromNow,WeekDay,Qty,Defect_0,Defect_1,Defect_2,Defect_3,Defect_4
953,89,1,0,2,2,12,5,994,2,1,2,0,4
492,89,1,1,1,3,12,5,929,0,0,0,1,0
95,89,0,1,2,3,12,5,941,0,1,0,1,2
679,89,1,1,0,3,12,5,926,0,6,3,0,0
78,89,0,0,2,2,12,5,933,1,0,0,2,3
141,89,0,1,0,4,12,5,790,1,1,1,1,2
835,89,1,1,0,1,12,5,906,0,1,4,0,1
343,89,0,1,2,1,12,5,888,0,0,0,1,3
807,88,1,1,1,3,12,4,879,2,3,1,3,0
970,88,0,1,1,3,12,4,1019,2,2,1,3,2


## EDA

In [38]:
df.groupby(['Site', 'WeeksFromNow']).sum()

DaysFromNow  Link  Shift  WeekDay    Qty  Defect_0  \
Site WeeksFromNow                                                       
0    0                      89    45     89       89  36677        79   
     1                     391    41     91      125  37541       110   
     2                     753    40    123      123  44766       142   
     3                    1059    54    108      135  44648       115   
     4                    1065    29     76      113  33940        88   
     5                    1254    28     89       99  32792       102   
     6                    1367    16     72      107  30072        73   
     7                    2144    38    124      135  40593       119   
     8                    2536    47    114      128  43957       116   
     9                    2972    43     88      137  46368       120   
     10                   3288    48    110      138  44319       123   
     11                   2326    28     78       93  29324        63   
     12                   3284    40    100       92  37987       120   
1    0                     134    49    119      134  40369       107   
     1                     301    27     77       84  31077        87   
     2                     689    34    101      115  40683       116   
     3                    1087    40    113      142  45903       120   
     4                    1354    34    117      122  44766       110   
     5                    1300    30     96      110  33945        93   
     6                    1427    25     69       83  33070        89   
     7                    2296    43    109      140  44463       127   
     8                    1774    27     86       94  29538        78   
     9                    2241    27     68       99  34750        96   
     10                   3197    42    111      117  42938       161   
     11                   3502    48    110      114  44586       109   
     12                   3016    41     86       76  36323        95   

                   Defect_1  Defect_2  Defect_3  Defect_4  
Site WeeksFromNow                                          
0    0                  122       132       103        97  
     1                   83        99       107       101  
     2                  103       142       124       120  
     3                  112       105       136       106  
     4                  115       101        81        83  
     5                   66        66        72        87  
     6                   82        90        84        90  
     7                  105       128       113        99  
     8                  127       149       137       138  
     9                  120       102       104       163  
     10                 134       116       108       107  
     11                  88        95        84        84  
     12                  94        76       130       102  
1    0                  134       126       139        85  
     1                   83        91        80        81  
     2                   98       107        73       111  
     3                  125       137       110       149  
     4                  129       140       140       115  
     5                  103       101        66        85  
     6                   87        90        78       111  
     7                  122       117       138       125  
     8                   72        94        91       100  
     9                   70        93        90       102  
     10                 146       134       125       133  
     11                 108       162       145       121  
     12                 117        85        85       115

##Algorithms

In [74]:
df_week = df[df['WeeksFromNow'] == 0].groupby(REF_GROUP + TRIGGER_GROUP).sum()[DATA_COLUMNS].reset_index()
df_week = df_week.melt(id_vars=df_week.columns[~df_week.columns.isin(DEFECT_COLUMNS)], value_vars=DEFECT_COLUMNS, var_name='DefectName', value_name='DefectQty')
df_week['Loss'] = df_week['DefectQty'] / df_week['Qty']
df_week

,Product,Site,Link,WeekDay,Shift,Qty,DefectName,DefectQty,Loss
0,0,0,0,0,2,1099,Defect_0,4,0.003640
1,0,0,0,0,5,919,Defect_0,1,0.001088
2,0,0,0,1,0,939,Defect_0,3,0.003195
3,0,0,1,1,2,925,Defect_0,1,0.001081
4,0,0,1,1,3,890,Defect_0,3,0.003371
...,...,...,...,...,...,...,...,...,...
345,1,1,2,2,0,1056,Defect_4,5,0.004735
346,1,1,2,2,1,863,Defect_4,2,0.002317
347,1,1,2,4,5,991,Defect_4,1,0.001009
348,1,1,2,5,3,1019,Defect_4,2,0.001963


In [75]:
df_quarter = df[df['WeeksFromNow'] != 0].groupby(REF_GROUP).sum()[DATA_COLUMNS].reset_index()
df_quarter = df_quarter.melt(id_vars=df_quarter.columns[~df_quarter.columns.isin(DEFECT_COLUMNS)], value_vars=DEFECT_COLUMNS, var_name='DefectName', value_name='DefectQty')
df_quarter['Loss'] = df_quarter['DefectQty'] / df_quarter['Qty']
df_quarter

,Product,Qty,DefectName,DefectQty,Loss
0,0,450686,Defect_0,764,0.001695
1,1,472797,Defect_0,840,0.001777
2,0,450686,Defect_1,877,0.001946
3,1,472797,Defect_1,823,0.001741
4,0,450686,Defect_2,794,0.001762
5,1,472797,Defect_2,861,0.001821
6,0,450686,Defect_3,781,0.001733
7,1,472797,Defect_3,790,0.001671
8,0,450686,Defect_4,841,0.001866
9,1,472797,Defect_4,766,0.001620


In [76]:
df_process = df_week.merge(df_quarter, on=REF_GROUP + ['DefectName'], how='left', suffixes=('_new', '_ref'))
df_process

,Product,Site,Link,WeekDay,Shift,Qty_new,DefectName,DefectQty_new,Loss_new,Qty_ref,DefectQty_ref,Loss_ref
0,0,0,0,0,2,1099,Defect_0,4,0.003640,450686,764,0.001695
1,0,0,0,0,5,919,Defect_0,1,0.001088,450686,764,0.001695
2,0,0,0,1,0,939,Defect_0,3,0.003195,450686,764,0.001695
3,0,0,1,1,2,925,Defect_0,1,0.001081,450686,764,0.001695
4,0,0,1,1,3,890,Defect_0,3,0.003371,450686,764,0.001695
...,...,...,...,...,...,...,...,...,...,...,...,...
345,1,1,2,2,0,1056,Defect_4,5,0.004735,472797,766,0.001620
346,1,1,2,2,1,863,Defect_4,2,0.002317,472797,766,0.001620
347,1,1,2,4,5,991,Defect_4,1,0.001009,472797,766,0.001620
348,1,1,2,5,3,1019,Defect_4,2,0.001963,472797,766,0.001620


In [77]:
def row_test (row,  suffixes=['_new', '_ref']):
    if row['Loss' + suffixes[0]] <= row['Loss' + suffixes[1]]:
        return np.nan
    observed_data = np.array([
        [row['Qty' + suffixes[0]], row['DefectQty' + suffixes[0]]],
        [row['Qty' + suffixes[1]], row['DefectQty' + suffixes[1]]]
    ])

    observed_data[:,0] = observed_data[:,0] - observed_data[:,1]
    # chi2, p_value, df, expected_freq = chi2_contingency(observed_data)
    # fisher's exact is the modern standard for 2x2 contingency table because improved computational capabilities as well as highly optimized factorial calculation
    odds_ratio, p_value = fisher_exact(observed_data, alternative='two-sided')
    return float(p_value)

In [78]:
df_process['p_value'] = df_process.apply(row_test, axis=1)
df_process

,Product,Site,Link,WeekDay,Shift,Qty_new,DefectName,DefectQty_new,Loss_new,Qty_ref,DefectQty_ref,Loss_ref,p_value
0,0,0,0,0,2,1099,Defect_0,4,0.003640,450686,764,0.001695,0.119520
1,0,0,0,0,5,919,Defect_0,1,0.001088,450686,764,0.001695,NaN
2,0,0,0,1,0,939,Defect_0,3,0.003195,450686,764,0.001695,0.215078
3,0,0,1,1,2,925,Defect_0,1,0.001081,450686,764,0.001695,NaN
4,0,0,1,1,3,890,Defect_0,3,0.003371,450686,764,0.001695,0.193859
...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,1,1,2,2,0,1056,Defect_4,5,0.004735,472797,766,0.001620,0.030500
346,1,1,2,2,1,863,Defect_4,2,0.002317,472797,766,0.001620,0.408157
347,1,1,2,4,5,991,Defect_4,1,0.001009,472797,766,0.001620,NaN
348,1,1,2,5,3,1019,Defect_4,2,0.001963,472797,766,0.001620,0.683119


In [79]:
df_triggers = df_process.dropna(subset=['p_value'])
df_triggers = df_triggers[df_triggers['p_value'] < P_THRESHOLD]
df_triggers

,Product,Site,Link,WeekDay,Shift,Qty_new,DefectName,DefectQty_new,Loss_new,Qty_ref,DefectQty_ref,Loss_ref,p_value
26,0,1,2,2,0,892,Defect_0,5,0.005605,450686,764,0.001695,0.019263
47,1,0,2,4,2,1095,Defect_0,6,0.005479,472797,840,0.001777,0.014778
73,0,0,1,1,2,925,Defect_1,5,0.005405,450686,877,0.001946,0.036561
90,0,1,1,5,5,751,Defect_1,5,0.006658,450686,877,0.001946,0.016874
107,1,0,1,3,3,866,Defect_1,5,0.005774,472797,823,0.001741,0.019021
120,1,0,2,6,3,959,Defect_1,5,0.005214,472797,823,0.001741,0.027849
242,1,0,0,5,2,1062,Defect_3,5,0.004708,472797,790,0.001671,0.034793
248,1,0,1,5,3,1902,Defect_3,7,0.003680,472797,790,0.001671,0.043736
256,1,0,2,2,5,1021,Defect_3,5,0.004897,472797,790,0.001671,0.030171
277,1,1,2,4,5,991,Defect_3,5,0.005045,472797,790,0.001671,0.027047


##Visualizations